In [5]:
from numpy import array, arctan2, sin, cos, eye
from scipy.linalg import block_diag, norm

# Time interval
dt = 1                        # second

# Initial position
pgb_g = array([[2.3],         # meter
               [3.7]])        # meter


# Observations

# Velocity
vgb_b = array([[12.8],        # meter/second
               [0]])          # meter/second

# Specific force
fgb_b = array([[1.8],         # meter/second^2
               [0.2]])        # meter/second^2

# Magnetometer
mgb_b = array([[120],         # uGauss
               [-180]])       # uGauss

# Gyroscope
ogb_b = array([[0],
               [0],
               [0.13]])       # radian/second


# Local functions

# Cross product matrix
def crossm(a):
    
    return array([[ 0, -a[2, 0], a[1, 0]],
                  [ a[2, 0], 0, -a[0, 0]],
                  [-a[1, 0], a[0, 0], 0]])

# Numerical integration
def numint(C, a):
    
    sigma = norm(a)
    B = crossm(a)
    
    # Compute C(t+dt)
    C = C@(eye(3) + (sin(sigma)/sigma)*B + ((1 - cos(sigma))/sigma**2)*B**2)
    
    return C


# Compute inital heading
psi = arctan2(-mgb_b[1, 0], mgb_b[0, 0])

# Compute Cb_g
Cb_g = array([[cos(psi), sin(psi)],
              [-sin(psi), cos(psi)]])

# Numerical integration
Cb_g_hat = numint(block_diag(Cb_g, 1), ogb_b*dt)
Cb_g_hat = Cb_g_hat[0:2, 0:2]

# Compute instantaneous acceleration in g-frame
agb_g = Cb_g_hat@fgb_b

# Compute velocity in g-frame
vgb_g = Cb_g_hat@vgb_b

# Compute position in g-frame
pgb_g = pgb_g + vgb_g*dt + 1/2*agb_g*dt**2
print(pgb_g)

[[11.54977283]
 [-6.58426965]]
